<a href="https://colab.research.google.com/github/ReevaKanakhara/EcoWatch-AI-Hacksagon/blob/main/EcoWatch_Dashboard_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q earthengine-api geemap

In [ ]:
import ee
import geemap

#manual link
ee.Authenticate(force=True)

In [ ]:
import ee

PROJECT_ID = 'ecowatch-ai-2026'

try:
    ee.Initialize(project=PROJECT_ID)
    print(f"Success! {PROJECT_ID} is now the active workspace for EcoWatch AI.")
except Exception as e:
    print(f"Initialization failed: {e}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("AI Libraries and GEE Backend Ready.")

In [ ]:
def fetch_historical_loss(lat, lon, size_px=64):
    #EcoWatch: 2000-2024 Timeline.

    # Define the 960m x 960m area (30m pixels x 64px = 1920m total area)
    region = ee.Geometry.Point([lon, lat]).buffer(size_px * 30 / 2).bounds()

    # Load the definitive 2000-2024 Forest Change Dataset
    gfc = ee.Image("UMD/hansen/global_forest_change_2024_v1_12")

    # 1. Background: Tree canopy cover in the year 2000 (0 to 100%)
    base_2000 = gfc.select('treecover2000').clip(region)

    # 2. Timeline: Pixels marked by the year of loss (1=2001 ... 24=2024)
    loss_year = gfc.select('lossyear').clip(region)

    # Bridge to AI: Convert to Numpy
    # normalize treecover to [0, 1] for the model
    base_np = geemap.ee_to_numpy(base_2000, region=region) / 100.0

    # Label: Binary mask (any loss between 2001-2024 = 1, else 0)
    loss_np = (geemap.ee_to_numpy(loss_year, region=region) > 0).astype(np.float32)

    return base_np, loss_np

In [ ]:
#BUILD ATTENTION U-NET MODEL
print("Building Attention U-Net for Forest Intelligence...")

class AttentionGate(layers.Layer):

    #Attention Gate for U-Net helps the model focus on relevant vegetation health features

    def __init__(self, filters, **kwargs):
        super(AttentionGate, self).__init__(**kwargs)
        self.filters = filters

    def build(self, input_shape):
        # Input shape: [(batch, h, w, c_g), (batch, h, w, c_x)]
        self.W_g = layers.Conv2D(self.filters, 1, padding='same', use_bias=True)
        self.W_x = layers.Conv2D(self.filters, 1, padding='same', use_bias=True)
        self.psi = layers.Conv2D(1, 1, padding='same', use_bias=True)
        self.relu = layers.ReLU()
        self.sigmoid = layers.Activation('sigmoid')
        super(AttentionGate, self).build(input_shape)

    def call(self, inputs):
        g, x = inputs  # g: gating signal, x: skip connection
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        psi = self.sigmoid(psi)
        return x * psi

    def get_config(self):
        config = super(AttentionGate, self).get_config()
        config.update({'filters': self.filters})
        return config

def conv_block(x, filters, kernel_size=3, name_prefix='conv'):
   #Convolutional block with BatchNorm for satellite data stability
    x = layers.Conv2D(filters, kernel_size, padding='same', activation='relu', name=f'{name_prefix}_conv1')(x)
    x = layers.BatchNormalization(name=f'{name_prefix}_bn1')(x)
    x = layers.Conv2D(filters, kernel_size, padding='same', activation='relu', name=f'{name_prefix}_conv2')(x)
    x = layers.BatchNormalization(name=f'{name_prefix}_bn2')(x)
    return x

def build_attention_unet(input_shape=(64, 64, 1)):

    #EcoWatch Attention U-Net optimized for single-channel NDVI inputs

    inputs = layers.Input(input_shape, name='input')

    #ENCODER
    enc1 = conv_block(inputs, 32, name_prefix='enc1')
    pool1 = layers.MaxPooling2D(2, name='pool1')(enc1)

    enc2 = conv_block(pool1, 64, name_prefix='enc2')
    pool2 = layers.MaxPooling2D(2, name='pool2')(enc2)

    enc3 = conv_block(pool2, 128, name_prefix='enc3')
    pool3 = layers.MaxPooling2D(2, name='pool3')(enc3)

    enc4 = conv_block(pool3, 256, name_prefix='enc4')
    pool4 = layers.MaxPooling2D(2, name='pool4')(enc4)

    #BOTTLENECK
    bottleneck = conv_block(pool4, 512, name_prefix='bottleneck')

    #DECODER WITH ATTENTION
    up1 = layers.UpSampling2D(2, name='up1')(bottleneck)
    up1 = layers.Conv2D(256, 2, padding='same', activation='relu', name='up1_conv')(up1)
    att1 = AttentionGate(256, name='att_gate1')([up1, enc4])
    merge1 = layers.Concatenate(name='merge1')([up1, att1])
    dec1 = conv_block(merge1, 256, name_prefix='dec1')

    up2 = layers.UpSampling2D(2, name='up2')(dec1)
    up2 = layers.Conv2D(128, 2, padding='same', activation='relu', name='up2_conv')(up2)
    att2 = AttentionGate(128, name='att_gate2')([up2, enc3])
    merge2 = layers.Concatenate(name='merge2')([up2, att2])
    dec2 = conv_block(merge2, 128, name_prefix='dec2')

    up3 = layers.UpSampling2D(2, name='up3')(dec2)
    up3 = layers.Conv2D(64, 2, padding='same', activation='relu', name='up3_conv')(up3)
    att3 = AttentionGate(64, name='att_gate3')([up3, enc2])
    merge3 = layers.Concatenate(name='merge3')([up3, att3])
    dec3 = conv_block(merge3, 64, name_prefix='dec3')

    up4 = layers.UpSampling2D(2, name='up4')(dec3)
    up4 = layers.Conv2D(32, 2, padding='same', activation='relu', name='up4_conv')(up4)
    att4 = AttentionGate(32, name='att_gate4')([up4, enc1])
    merge4 = layers.Concatenate(name='merge4')([up4, att4])
    dec4 = conv_block(merge4, 32, name_prefix='dec4')

    #OUTPUT: Binary Forest Loss Prediction
    outputs = layers.Conv2D(1, 1, activation='sigmoid', dtype='float32', name='output')(dec4)

    model = models.Model(inputs=inputs, outputs=outputs, name='EcoWatch_AttentionUNet')
    return model

#COMPILE MODEL
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def combined_loss(y_true, y_pred):
    focal = tf.keras.losses.BinaryFocalCrossentropy(alpha=0.25, gamma=2.0)(y_true, y_pred)
    dice = 1 - dice_coefficient(y_true, y_pred)
    return 0.5 * focal + 0.5 * dice

model = build_attention_unet()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=combined_loss,
    metrics=['accuracy', tf.keras.metrics.Recall(name='recall'), dice_coefficient]
)

print("Attention U-Net Model ready for EcoWatch training loop.")

In [ ]:
from skimage.transform import resize

# Target hotspots for high-impact signatures
hotspots = [
    [22.91, 92.95], [23.10, 92.80], [22.80, 93.10], # Mizoram
    [25.55, 93.68], [25.70, 93.80], [25.40, 93.50], # Nagaland
    [-11.40, -58.57], [-11.50, -58.70], [-11.30, -58.40], # Amazon
    [-0.50, 20.00], [-0.60, 20.10], [-0.40, 19.90]  # Congo
]

X_train_list, y_train_list = [], []

print("Harvesting & Standardizing Expanded Training Set...")

for lat, lon in hotspots:
    try:
        # Pull 24-year historical data
        img, loss = fetch_historical_loss(lat, lon)

        # Standardize shape to exactly 64x64x1
        img_std = resize(img, (64, 64, 1), preserve_range=True)
        loss_std = resize(loss, (64, 64, 1), preserve_range=True)

        # Augmentation: Add 4 rotations of each patch
        for r in [0, 1, 2, 3]:
            X_train_list.append(np.rot90(img_std, r))
            y_train_list.append(np.rot90(loss_std, r))

    except Exception as e:
        continue

# Convert to final Numpy arrays
X_train = np.array(X_train_list)
y_train = np.array(y_train_list)

print(f"Final Training Set: {X_train.shape[0]} patches ready!")

In [ ]:
print("Launching Attention U-Net Training...")

# Training for 50 epochs to allow convergence
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=4,
    verbose=1
)

print("\nTraining Complete! The model is now tuned for global biomes.")

In [ ]:
print("Generating 24-Year Visual Validation Report...\n")

target_names = ["Mizoram, India", "Nagaland, India", "Amazon, Brazil", "Congo Basin, DRC"]
demo_locations = [[22.91, 92.95], [25.55, 93.68], [-11.40, -58.57], [-0.50, 20.00]]

fig, axes = plt.subplots(4, 3, figsize=(18, 20))

for i, loc in enumerate(demo_locations):
    # Retrieve and standardize input for visualization
    img, loss = fetch_historical_loss(loc[0], loc[1])
    img = resize(img, (64, 64, 1), preserve_range=True)

    # Run prediction
    pred = model.predict(np.expand_dims(img, axis=0), verbose=0)[0]

    # 1. Forest Canopy in 2000
    axes[i, 0].imshow(img.squeeze(), cmap='Greens')
    axes[i, 0].set_title(f"{target_names[i]}\nForest Canopy (2000)", fontweight='bold')
    axes[i, 0].axis('off')

    # 2. Actual Historical Loss (2000-2024)
    axes[i, 1].imshow(loss.squeeze(), cmap='Reds')
    axes[i, 1].set_title("Actual Loss (2000-2024)", fontweight='bold')
    axes[i, 1].axis('off')

    # 3. AI Prediction (Thresholded at 0.5 for sharpness)
    axes[i, 2].imshow(pred.squeeze() > 0.5, cmap='OrRd')
    axes[i, 2].set_title("EcoWatch AI Prediction", fontweight='bold')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
print("Generating EcoWatch AI Final Impact Report (2000-2024)...")

# Calculate statistics for each of your 4 official targets
for i in range(len(X_final)):
    # Calculate total forest pixels in 2000 (Baseline)
    # We use a threshold of 0.3 to define 'forested' areas in the year 2000
    forest_2000_pixels = np.sum(X_final[i] > 0.3)

    # Calculate pixels lost by 2024
    loss_pixels = np.sum(y_final[i] > 0)

    # Calculate AI's predicted loss pixels
    pred = model.predict(X_final[i:i+1], verbose=0)[0]
    pred_loss_pixels = np.sum(pred > 0.5)

    # Percentage calculation
    actual_loss_pct = (loss_pixels / forest_2000_pixels) * 100 if forest_2000_pixels > 0 else 0
    ai_confidence = (1 - abs(pred_loss_pixels - loss_pixels) / loss_pixels) * 100 if loss_pixels > 0 else 100

    print(f"REGION: {target_names[i]}")
    print(f"• Baseline Forest Density (2000): {np.mean(X_final[i])*100:.1f}%")
    print(f"• Actual Forest Loss (2000-2024): {actual_loss_pct:.1f}% of canopy")
    print(f"• AI Prediction Accuracy:        {ai_confidence:.1f}%")
    print("-" * 50)

# Final Summary
eval_metrics = model.evaluate(X_train, y_train, verbose=0)
print(f"\nFINAL MODEL STANDINGS:")
print(f"• Global Accuracy: {eval_metrics[1]*100:.2f}%")
print(f"• Dice Coefficient: {eval_metrics[3]:.4f}")
print(f"• Recall Rate:      {eval_metrics[2]*100:.2f}%")

In [ ]:
import os

drive_folder = '/content/drive/MyDrive/EcoWatch_Hacksagon'

if not os.path.exists(drive_folder):
    os.makedirs(drive_folder)
    print(f"Created new directory: {drive_folder}")

model_filename = 'ecowatch_v1_final.keras'
save_path = os.path.join(drive_folder, model_filename)

model.save(save_path)

print(f"Model saved at {save_path}")